In [1]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import joblib
from datetime import datetime
import concurrent

BASE_DIR = "../"
PASQUINIS_PATH = BASE_DIR + "traces-netsoft-2017"
DATE = datetime.now().isoformat(timespec='seconds')
BASE_RESULTS_PATH = f'./resultados_pre_pesquisa/{DATE}'
MODELS_DIR = "models/"
MODELS_PATH_PREFIX = f'models/{DATE}_'

TRACES=[
    #"KV-BothApps-FlashcrowdLoad",
    #"KV-BothApps-PeriodicLoad",
    #"KV-SingleApp-FlashcrowdLoad",
    #"KV-SingleApp-PeriodicLoad",
    #"VoD-BothApps-FlashcrowdLoad",
    #"VoD-BothApps-PeriodicLoad",
    #"VoD-SingleApp-FlashcrowdLoad",
    "VoD-SingleApp-PeriodicLoad"]

NROWS = None
PERSIST = False
Y_METRIC = 'DispFrames'
VOD_SINGLEAPP_PERIODIC_LOAD_PATH = f'{PASQUINIS_PATH}/VoD-SingleApp-PeriodicLoad'

def nmae(y_pred, y_test):
    return abs(y_pred - y_test).mean() / y_test.mean()

if PERSIST:
    results_path = f'{BASE_RESULTS_PATH}'
    try:
        os.makedirs(results_path)
        os.makedirs(MODELS_DIR)
    except FileExistsError:
        print("Já criado diretório...")

In [2]:
y_dataset = pd.read_csv(f'{VOD_SINGLEAPP_PERIODIC_LOAD_PATH}/Y.csv', header=0, nrows=NROWS, index_col=0, usecols=['TimeStamp', 'DispFrames'], low_memory=True).apply(pd.to_numeric, errors='coerce').fillna(0)

x_files = ['X_cluster.csv', 'X_flow.csv', 'X_port.csv']

x_trace = pd.DataFrame()

for x_file in x_files:
    read_dataset = pd.read_csv(f'{VOD_SINGLEAPP_PERIODIC_LOAD_PATH}/{x_file}', 
                              header=0, index_col=0, low_memory=True, nrows=NROWS).apply(pd.to_numeric, errors='coerce').fillna(0)
    if len(x_trace.columns) != 0:
        x_trace.merge(read_dataset, how="inner",
            on="TimeStamp", copy=False)
    else:
        x_trace = read_dataset



In [3]:
X_train, X_test, y_train, y_test = train_test_split(x_trace, y_dataset, test_size=0.7, random_state=42)


In [8]:
import time

In [5]:

regression_tree = DecisionTreeRegressor() 

tempo_reg_tree = time.time()
regression_tree.fit(X_train, y_train)
tempo_reg_tree = time.time() - tempo_reg_tree

if PERSIST:
    joblib.dump(regression_tree, f'{MODELS_PATH_PREFIX}_model_regression-tree.sav')

print(f'Training Time Reg Tree: {tempo_reg_tree}')
pred_reg_tree = regression_tree.predict(X_test)
print(f'NMAE: {nmae(pred_reg_tree, y_test["DispFrames"])}')

Training Time Reg Tree: 32.21946382522583
NMAE: 0.10037706997515161


In [6]:
regr_random_forest = RandomForestRegressor(n_estimators=120, random_state=42, n_jobs=-1)

tempo_random_forest = time.time()
regr_random_forest.fit(X_train, y_train)
tempo_random_forest = time.time() - tempo_random_forest

if PERSIST:
    joblib.dump(regr_random_forest, f'{MODELS_PATH_PREFIX}_model_random-forest.sav')

print(f'Training Time Random Forest: {tempo_random_forest}')
pred_random_forest = regr_random_forest.predict(X_test)
print(f'NMAE: {nmae(pred_random_forest, y_test["DispFrames"])}')

/home/davi/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training Time Random Forest: 308.4941403865814
NMAE: 0.0934177251849149


In [7]:
print(f'NMAE: {nmae(pred_random_forest, y_test["DispFrames"])}')

NMAE: 0.0934177251849149


In [8]:
## para X_flow ou x_port

x_file = 'X_flow.csv' 

print(f'Only dataset {x_file}')
x_trace_minimal_flow = pd.DataFrame()

x_trace_minimal_flow = pd.read_csv(f'{VOD_SINGLEAPP_PERIODIC_LOAD_PATH}/{x_file}', 
                          header=0, index_col=0, low_memory=True, nrows=NROWS).apply(pd.to_numeric, errors='coerce').fillna(0)

X_train_flow, X_test_flow, y_train_flow, y_test_flow = train_test_split(x_trace_minimal_flow, y_dataset, test_size=0.7, random_state=42)

regression_tree_flow = DecisionTreeRegressor() 

tempo_reg_tree_flow = time.time()
regression_tree_flow.fit(X_train_flow, y_train_flow)
tempo_reg_tree_flow = time.time() - tempo_reg_tree_flow

if PERSIST:
    joblib.dump(regression_tree_flow, f'{MODELS_PATH_PREFIX}_model_regression-tree_for-{x_file}.sav')

print(f'Training Time only {x_file} Reg Tree: {tempo_reg_tree_flow}')
pred_reg_tree_flow = regression_tree_flow.predict(X_test_flow)
print(f'NMAE only {x_file}: {nmae(pred_reg_tree_flow, y_test_flow["DispFrames"])}')

regr_random_forest_flow = RandomForestRegressor(n_estimators=120, random_state=42, n_jobs=-1)

tempo_random_forest_flow = time.time()
regr_random_forest_flow.fit(X_train_flow, y_train_flow)
tempo_random_forest_flow = time.time() - tempo_random_forest_flow

if PERSIST:
    joblib.dump(regr_random_forest_flow, f'{MODELS_PATH_PREFIX}_model_random-forest_for-{x_file}.sav')

print(f'Training Time Random Forest only {x_file}: {tempo_random_forest_flow}')
pred_random_forest_flow = regr_random_forest_flow.predict(X_test_flow)
print(f'NMAE only {x_file}: {nmae(pred_random_forest_flow, y_test_flow["DispFrames"])}')



Only dataset X_flow.csv
Training Time only X_flow.csv Reg Tree: 0.27962470054626465
NMAE only X_flow.csv: 0.11457965289482779


/home/davi/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training Time Random Forest only X_flow.csv: 3.1818175315856934
NMAE only X_flow.csv: 0.10733201366994324


In [9]:
#'X_port.csv'

x_file = 'X_port.csv' 

print(f'Only dataset {x_file}')
x_trace_minimal_port = pd.DataFrame()

x_trace_minimal_port = pd.read_csv(f'{VOD_SINGLEAPP_PERIODIC_LOAD_PATH}/{x_file}', 
                          header=0, index_col=0, low_memory=True, nrows=NROWS).apply(pd.to_numeric, errors='coerce').fillna(0)

X_train_port, X_test_port, y_train_port, y_test_port = train_test_split(x_trace_minimal_port, y_dataset, test_size=0.7, random_state=42)

regression_tree_port = DecisionTreeRegressor() 

tempo_reg_tree_port = time.time()
regression_tree_port.fit(X_train_port, y_train_port)
tempo_reg_tree_port = time.time() - tempo_reg_tree_port

if PERSIST:
    joblib.dump(regression_tree_port, f'{MODELS_PATH_PREFIX}_model_regression-tree_for-{x_file}.sav')

print(f'Training Time only {x_file} Reg Tree: {tempo_reg_tree_port}')
pred_reg_tree_port = regression_tree_port.predict(X_test_port)
print(f'NMAE only {x_file}: {nmae(pred_reg_tree_port, y_test_port["DispFrames"])}')

regr_random_forest_port = RandomForestRegressor(n_estimators=120, random_state=42, n_jobs=-1)

tempo_random_forest_port = time.time()
regr_random_forest_port.fit(X_train_port, y_train_port)
tempo_random_forest_port = time.time() - tempo_random_forest_port

if PERSIST:
    joblib.dump(regr_random_forest_port, f'{MODELS_PATH_PREFIX}_model_random-forest_for-{x_file}.sav')

print(f'Training Time Random Forest only {x_file}: {tempo_random_forest_port}')
pred_random_forest_port = regr_random_forest_port.predict(X_test_port)
print(f'NMAE only {x_file}: {nmae(pred_random_forest_port, y_test_port["DispFrames"])}')



Only dataset X_port.csv
Training Time only X_port.csv Reg Tree: 2.4806292057037354
NMAE only X_port.csv: 0.11184903299796696


/home/davi/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training Time Random Forest only X_port.csv: 27.997825145721436
NMAE only X_port.csv: 0.10501668143668873


In [34]:
#minimal dataset creation
## hypothesis, is selecting 12 most important features suficcient?
import warnings 

warnings.filterwarnings('ignore')

x_trace_minimal = pd.DataFrame()
old_reg_tree_nmae = 1
old_random_forest_nmae = 1
columns_with_sufficient_corr = []

x_trace_corrwith_y_metric = abs(x_trace.corrwith(y_dataset[Y_METRIC]))
x_trace_corrwith_y_metric.fillna(0, inplace=True)
x_trace_corrwith_y_metric.sort_values(inplace=True, ascending=False)

In [43]:
x_trace_corrwith_y_metric.index[0:12]

Index(['3_ldavg.1', '3_orq.s', '4_ldavg.1', '3_file.nr', '4_kbcommit',
       '4_X..commit', '4_plist.sz', '3_runq.sz', '4_file.nr', '3_totsck',
       '3_plist.sz', '2_file.nr'],
      dtype='object')

In [46]:
#training only with reg tree

x_trace_minimal_reg_tree = pd.DataFrame()
old_reg_tree_nmae = 1

for feature in x_trace_corrwith_y_metric.index:
    x_trace_minimal_reg_tree[feature] = x_trace[[feature]].copy()
    
    X_train_minimal , X_test_minimal, y_train_minimal, y_test_minimal = train_test_split(x_trace_minimal_reg_tree, y_dataset, test_size=0.7, random_state=42)
    
    regression_tree_minimal = DecisionTreeRegressor() 
    regression_tree_minimal.fit(X_train_minimal, y_train_minimal)
    pred_reg_tree_minimal = regression_tree_minimal.predict(X_test_minimal)
    
    new_nmae_reg_tree = nmae(pred_reg_tree_minimal, y_test_minimal["DispFrames"])
     
    if old_reg_tree_nmae >= new_nmae_reg_tree:
        old_reg_tree_nmae = new_nmae_reg_tree
    else:
        x_trace_minimal_reg_tree.drop([feature], axis=1, inplace=True)

print(x_trace_corrwith_y_metric.index[0:12])
print(x_trace_minimal_reg_tree.columns)
print(len(x_trace_minimal_reg_tree.columns))

X_train_minimal , X_test_minimal, y_train_minimal, y_test_minimal = train_test_split(x_trace_minimal_reg_tree, y_dataset, test_size=0.7, random_state=42)

regression_tree_minimal = DecisionTreeRegressor() 

tempo_reg_tree_minimal = time.time()
regression_tree_minimal.fit(X_train_minimal, y_train_minimal)
tempo_reg_tree_minimal = time.time() - tempo_reg_tree_minimal


pred_reg_tree_minimal = regression_tree_minimal.predict(X_test_minimal)

new_nmae_reg_tree = nmae(pred_reg_tree_minimal, y_test_minimal["DispFrames"])

print(f'MINIMAL NMAE REG_TREE {new_nmae_reg_tree}\n TRAINING TIME MINIMAL REG_TREE {tempo_reg_tree_minimal}')

random_forest_minimal = RandomForestRegressor(n_estimators=120, random_state=42, n_jobs=-1)

tempo_random_forest_minimal = time.time()
random_forest_minimal.fit(X_train_minimal, y_train_minimal)
tempo_random_forest_minimal = time.time() - tempo_random_forest_minimal

pred_random_forest_minimal = random_forest_minimal.predict(X_test_minimal)

new_nmae_random_forest = nmae(pred_random_forest_minimal, y_test_minimal["DispFrames"])

print(f'MINIMAL NMAE RANDOM_FOREST {new_nmae_random_forest}\n TRAINING TIME MINIMAL RANDOM FOREST {tempo_random_forest_minimal}')


Index(['3_ldavg.1', '3_orq.s', '4_ldavg.1', '3_file.nr', '4_kbcommit',
       '4_X..commit', '4_plist.sz', '3_runq.sz', '4_file.nr', '3_totsck',
       '3_plist.sz', '2_file.nr'],
      dtype='object')
Index(['3_ldavg.1', '4_ldavg.1', '2_ldavg.1', '4_ldavg.5', '2_kbinact',
       '1_inode.nr', '1_ldavg.15', '3_cpu8_.iowait', '3_cpu6_.iowait',
       '4_cpu22_.iowait', '2_cpu7_.iowait', '1_kbcommit', '0_cpu17_.iowait',
       '5_cpu22_.irq', '0_cpu14_.soft', '4_kbinact', '3_all_..nice',
       '1_cpu17_.sys', '2_cpu21_.soft', '1_cpu15_.idle', '3_cpu8_MHz',
       '5_i804_intr.s', '4_i625_intr.s'],
      dtype='object')
23
MINIMAL NMAE REG_TREE 0.06111487601869711
 TRAINING TIME MINIMAL REG_TREE 0.10103774070739746
MINIMAL NMAE RANDOM_FOREST 0.06803638348769454
 TRAINING TIME MINIMAL RANDOM FOREST 2.217909574508667


```
Index(['3_ldavg.1', '3_orq.s', '4_ldavg.1', '3_file.nr', '4_kbcommit',
       '4_X..commit', '4_plist.sz', '3_runq.sz', '4_file.nr', '3_totsck',
       '3_plist.sz', '2_file.nr'],
      dtype='object')
Index(['3_ldavg.1', '4_ldavg.1', '2_ldavg.1', '4_ldavg.5', '2_kbinact',
       '1_inode.nr', '1_ldavg.15', '3_cpu8_.iowait', '3_cpu6_.iowait',
       '4_cpu22_.iowait', '2_cpu7_.iowait', '1_kbcommit', '0_cpu17_.iowait',
       '5_cpu22_.irq', '0_cpu14_.soft', '4_kbinact', '3_all_..nice',
       '1_cpu17_.sys', '2_cpu21_.soft', '1_cpu15_.idle', '3_cpu8_MHz',
       '5_i804_intr.s', '4_i625_intr.s'],
      dtype='object')
23
MINIMAL NMAE REG_TREE 0.06111487601869711
 TRAINING TIME MINIMAL REG_TREE 0.10103774070739746
MINIMAL NMAE RANDOM_FOREST 0.06803638348769454
 TRAINING TIME MINIMAL RANDOM FOREST 2.217909574508667
```

Como dá pra ver, overfittou só listando, acho que não vai ter como fugir do stepwise

[('3_ldavg.1', 0.45847509678729365), ('3_orq.s', 0.45168820142684335), ('4_ldavg.1', 0.45024980784563573), ('3_file.nr', 0.44996068760540914), ('4_kbcommit', 0.4495659506994577), ('4_X..commit', 0.4495653109214851), ('4_plist.sz', 0.44897666850862666), ('3_runq.sz', 0.4485224345481781), ('4_file.nr', 0.4481894909360291), ('3_totsck', 0.44711913805990866), ('3_plist.sz', 0.4464711658760101), ('2_file.nr', 0.44642624088831534), ('3_X..commit', 0.4463452055931451), ('3_kbcommit', 0.4463419662727667), ('2_totsck', 0.44614714805115585), ('3_tcpsck', 0.44611022423215735), ('5_totsck', 0.44595352223880963), ('5_tcpsck', 0.44591287874574614), ('2_plist.sz', 0.4454382450324796), ('2_X..commit', 0.4446645216525189), ('2_kbcommit', 0.44466048377267325), ('2_tcpsck', 0.44464193877943026), ('5_inode.nr', 0.4444987551985621), ('4_tcpsck', 0.4444239395014656), ('4_runq.sz', 0.44079099908420044), ('2_ldavg.1', 0.4322258169862442), ('3_sum_intr.s', 0.4176936249566355), ('4_pgfree.s', 0.4165186152672803

ValueError: Found input variables with inconsistent numbers of samples: [0, 37036]

In [ ]:
# REPLICAÇÃO POR SWITCH
## 

In [ ]:
# para os demais

TRACES=[
    #"KV-BothApps-FlashcrowdLoad",
    #"KV-BothApps-PeriodicLoad",
    #"KV-SingleApp-FlashcrowdLoad",
    #"KV-SingleApp-PeriodicLoad",
    "VoD-BothApps-FlashcrowdLoad",
    "VoD-BothApps-PeriodicLoad",
    "VoD-SingleApp-FlashcrowdLoad",
    "VoD-SingleApp-PeriodicLoad"]

for trace in TRACES:
    print(trace)
    y_dataset_trace = pd.read_csv(f'{PASQUINIS_PATH}/{trace}/Y.csv', header=0, nrows=NROWS, index_col=0, usecols=['TimeStamp', 'DispFrames'], low_memory=True).apply(pd.to_numeric, errors='coerce').fillna(0)

    for x_files in [['X_cluster.csv', 'X_flow.csv', 'X_port.csv'], ['X_port.csv'], ['X_flow.csv']]:
        print(x_files)
        x = pd.DataFrame()

        for x_file in x_files:
            read_dataset = pd.read_csv(f'{PASQUINIS_PATH}/{trace}/{x_file}', 
                                      header=0, index_col=0, low_memory=True, nrows=NROWS).apply(pd.to_numeric, errors='coerce').fillna(0)
            if len(x_trace.columns) != 0:
                x_trace.merge(read_dataset, how="inner",
                    on="TimeStamp", copy=False)
            else:
                x_trace = read_dataset

        x_training, x_testing, y_training, y_testing = train_test_split(x, y_dataset_trace, test_size=0.7, random_state=42)

        print('full_x')
        print('regression_method, nmae, training_time')

        regression_tree_trace = DecisionTreeRegressor() 
        tempo_reg_tree_trace = time.time()
        regression_tree_trace.fit(x_training, y_training)
        tempo_reg_tree_trace = time.time() - tempo_reg_tree_trace
        pred_reg_tree_trace = regression_tree_trace.predict(x_testing)

        print(f'Reg_Tree, {nmae(pred_reg_tree_trace, y_testing["DispFrames"])}, {tempo_reg_tree_trace}')

        regr_random_forest_trace = RandomForestRegressor(n_estimators=120, random_state=42, n_jobs=-1)
        tempo_random_forest_trace = time.time()
        regr_random_forest_trace.fit(x_training, y_training)
        tempo_random_forest_trace = time.time() - tempo_random_forest
        pred_random_forest_trace = regr_random_forest_trace.predict(x_testing)

        print(f'Rand_For, {nmae(pred_random_forest_trace, y_testing["DispFrames"])}, {tempo_random_forest_trace}')



VoD-BothApps-FlashcrowdLoad
['X_cluster.csv', 'X_flow.csv', 'X_port.csv']
